In [3]:
import requests
from bs4 import BeautifulSoup
import time

# Define Target url

In [4]:
# pubmed url
# search
# https://www.biomedcentral.com/search?searchType=publisherSearch&sort=Relevance&query=Colorectal+cancer&page=2
# article
# https://hccpjournal.biomedcentral.com/articles/10.1186/1897-4287-8-S1-P16
base_url = 'https://cancerandmetabolism.biomedcentral.com/'
search_url = 'https://www.biomedcentral.com/search?searchType=publisherSearch&sort=Relevance&query='
page_url = '&page='

# define scraping tags

In [11]:
# article name in search page
search_article_name_tag = 'h3'
search_article_name_class = 'c-listing__title'

# abstract
abstract_tag = 'div'
abstract_class = 'c-article-section__content'

keyword = 'Colorectal+cancer'

# define scraping range

In [12]:
# scrap from page start_page_num to page to_page_num
start_page_num = 1
to_page_num = 500

# save paths
fpath = 'data/'

In [13]:
import pandas as pd

In [41]:
# start scraping
page = start_page_num

complete_cnt = 0

for i in range(start_page_num,to_page_num+1):
    rq = requests.get(search_url + keyword + page_url + str(i)) # get search page
    soup = BeautifulSoup(rq.text, 'lxml')
    articles = soup.find_all(search_article_name_tag, class_=search_article_name_class)

    # print(articles)

    title = []
    abstract = [[], [], [], []]
    for art in articles:
        # print("https:" + i.a.get('href'))
        title.append(art.text.strip()) # title
        # get abstract page
        rq = requests.get("https:" + art.a.get('href'))
        time.sleep(0.1)
        soup = BeautifulSoup(rq.text, 'lxml')

        # not guarentee every doc has background method result conclusion
        try:
            temp_abstract = []
            for i in range(1,5,1):
                # get abstract tag
                _abstract = soup.find(abstract_tag, class_=abstract_class,
                                        id="Sec{}-content".format(i))
                temp_abstract.append(_abstract.text)
            
            for j, w in enumerate(temp_abstract):
                abstract[j].append(w)

            complete_cnt += 1
        except:
            # for i in abstract:
            #     i.append('')
            title.pop()

    # save as csv
    d = {'title' : title, 'Background' : abstract[0], 'Methods': abstract[1], 'Results': abstract[2], 'Conclusion': abstract[3]}
    one_page_df = pd.DataFrame(d)
    one_page_df.to_csv(fpath + str(page) + '.csv')

    page += 1
    
    print(complete_cnt)
    time.sleep(0.7)

7
20
29


KeyboardInterrupt: 